In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, RobustScaler
import pandas_profiling as pp
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error,classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/kaggle/input/heart-disease-uci/heart.csv')
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.describe()

Total unique values in target column

In [ ]:
df['target'].value_counts()

Our dataset is balanced

Shape of dataset

In [ ]:
df.shape

Let's check null value

In [ ]:
df.isnull().sum()

There are no null values we are good to go

In [ ]:
plt.figure(figsize = (8, 8))
sns.distplot(df['age'], color='blue')
plt.title('age distribution', fontsize = 14)
plt.show()

In [ ]:
print("Skewness of age : ", df['age'].skew())

Age is negative skewd

In [ ]:
print("Maximum age : ", df['age'].max())
print("Minimum age : ", df['age'].min())
print("Average age : ", round(df['age'].mean()))

In [ ]:
df['sex'].value_counts()

In [ ]:
df.groupby(['sex','target'])['target'].count()

In [ ]:
vgames_profile = ProfileReport(df, title='Heart Disease')
vgames_profile

Unique value count in cp columns

In [ ]:
df['cp'].value_counts()

In [ ]:
plt.figure(figsize = (8, 6))
sns.histplot(df['cp'])
plt.title('cp histplot', fontsize = 15)
plt.show()

In [ ]:
f, ax = plt.subplots(1,3, figsize=(24, 6))

sns.histplot(df['trestbps'], ax=ax[0])
ax[0].set_title("trestbps histplot")

sns.distplot(df['trestbps'], ax=ax[1])
ax[1].set_title('trestbps distplot')

sns.stripplot(x=df['target'], y=df['trestbps'], ax=ax[2])
ax[2].set_title('trestbps vs target')

plt.grid()

plt.show()

In [ ]:
print("Skewness of trestbps : ", df['trestbps'].skew())

trestbps is positive skewd

In [ ]:
f, ax = plt.subplots(1,3, figsize=(24, 6))

sns.histplot(df['chol'], ax=ax[0])
ax[0].set_title("Chol histplot")

sns.distplot(df['chol'], ax=ax[1])
ax[1].set_title('Chol distplot')

sns.stripplot(x=df['target'], y=df['chol'], ax=ax[2])
ax[2].set_title('chol vs target')

plt.grid()

plt.show()

In [ ]:
print("Skewness of chol : ",df['chol'].skew())

chol is positive skewed

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(df['fbs'])
plt.title('fbs',fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(df['restecg'])
plt.title('restecg',fontsize=14)
plt.show()

In [ ]:
f, ax = plt.subplots(1,3, figsize=(24, 6))

sns.histplot(df['thalach'], ax=ax[0])
ax[0].set_title("thalach histplot")

sns.distplot(df['thalach'], ax=ax[1])
ax[1].set_title('thalach distplot')

sns.stripplot(x=df['target'], y=df['thalach'], ax=ax[2])
ax[2].set_title('thalach vs target')

plt.grid()

plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(df['exang'])
plt.title('exang',fontsize=14)
plt.show()

In [ ]:
f, ax = plt.subplots(1,3, figsize=(24, 6))

sns.histplot(df['oldpeak'], ax=ax[0])
ax[0].set_title("oldpeak histplot")

sns.distplot(df['oldpeak'], ax=ax[1])
ax[1].set_title('oldpeak distplot')

sns.stripplot(x=df['target'], y=df['oldpeak'], ax=ax[2])
ax[2].set_title('oldpeak vs target')

plt.grid()

plt.show()


In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(df['slope'])
plt.title('slope',fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(df['ca'])
plt.title('ca',fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(df['thal'])
plt.title('thal',fontsize=14)
plt.show()

Age column is continuous variable let's try convert in categorical

In [ ]:
df['age'].describe()

In [ ]:
df.loc[(df['age']>=29.0) & (df['age']<=47.50), 'age'] = 0
df.loc[(df['age']>47.50) & (df['age']<=55.0), 'age'] = 1
df.loc[(df['age']>55.0) & (df['age']<61.0), 'age'] = 2
df.loc[(df['age']>=61.0) & (df['age']<=77), 'age'] = 3

In [ ]:
df['age'].value_counts()

Convert some columns to normal distribution

In [ ]:
df['trestbps'] = (df['trestbps'] - df['trestbps'].mean()) / df['trestbps'].std()
df['chol'] = (df['chol'] - df['chol'].mean()) / df['chol'].std()
df['thalach'] = (df['thalach'] - df['thalach'].mean()) / df['thalach'].std()
df['oldpeak'] = (df['oldpeak'] - df['oldpeak'].mean()) / df['oldpeak'].std()

In [ ]:
df.head()

Slit into X and y

In [ ]:
X = df.drop('target',axis=1)
y = df['target']

X = RobustScaler().fit_transform(X)

Split into train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

**Logistic Regression**

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print("Logistic Regression score : ", lr.score(X_test, y_test))
print("classificatioon report")
print(classification_report(y_pred, y_test))

**Decison tree classifier**

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print("Decision tree classifier score : ", dt.score(X_test, y_test))
print("classificatioon report")
print(classification_report(y_pred, y_test))

**Random forest classifier**

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print("Random forest classifier score : ", rf.score(X_test, y_test))
print("classificatioon report")
print(classification_report(y_pred, y_test))

**Gradient boosting classifier**

In [ ]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
     max_depth=1, random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)
y_pred = clf.predict(X_test)
print("Gradient boosting classifier score : ", clf.score(X_test, y_test))
print("classificatioon report")
print(classification_report(y_pred, y_test))

**Kneighbours classifier**

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
neigh.score(X_test, y_test)
y_pred = neigh.predict(X_test)
print("Gradient boosting classifier score : ", neigh.score(X_test, y_test))
print("classificatioon report")
print(classification_report(y_pred, y_test))

**Logistic regression and Random forest classifier perform best accuracy in this dataset**

**If you like this dont't forget to upwote it**